In [1]:
### SETUP CELL - RUN THIS FIRST
!pip install rdflib
import rdflib
from rdflib import Graph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 9.0 MB/s eta 0:00:00


In [2]:
path_to_ex1 = "https://raw.githubusercontent.com/IDA-FBK/mental-health-ontology/refs/heads/main/examples/ex1-manual.ttl"
ex1_graph = Graph()
ex1_graph.parse(path_to_ex1, format="ttl")

path_to_ex2 = "https://raw.githubusercontent.com/IDA-FBK/mental-health-ontology/refs/heads/main/examples/ex2-automatic.ttl"
ex2_graph = Graph()
ex2_graph.parse(path_to_ex2, format="ttl")

<Graph identifier=Nd0a446a05c63492d8e90a0d383343f1d (<class 'rdflib.graph.Graph'>)>

In [3]:
## CQ ##
# Which data sources did human experts use to detect depression?

q1 = ex1_graph.query(
"""
  PREFIX ex1:<https://w3id.org/miao/ex1#>
  SELECT DISTINCT ?expertLabel ?dataSourceLabel
  WHERE {
    ?detection a miao:ManualMentalIllnessesDetection ;
            miao:hasInputData ?data ;
            prov:wasAssociatedWith ?expert ;
            prov:generated ?mentalIllnessesSet .
    ?data a mls:Dataset ;
        rdfs:label ?dataSourceLabel .
    ?expert a prov:Person ;
        rdfs:label ?expertLabel .
    ?mentalIllnessesSet a miao:MentalIllnessesSet ;
        miao:hasMentalIllness ?mentalIllness .
    ex1:Depression rdfs:subClassOf miao:MentalIllness .
    ?depressionInstance a ex1:Depression ;
  }
"""
)
for result in q1:
    print("Expert {} use data source '{}' to detect depression".format(result[0], result[1]))

Expert Dr. Laura García, PhD use data source 'Self-reported depression questionnaire responses' to detect depression
Expert Dr. Laura García, PhD use data source 'Therapist session notes' to detect depression


In [4]:
## CQ ##
# Which mental illness schemas do experts reference during manual analysis?

q2 = ex1_graph.query(
"""
  SELECT DISTINCT ?expertLabel ?schemaTitle
  WHERE {
    ?detection a miao:ManualMentalIllnessesDetection ;
              prov:wasAssociatedWith ?expert ;
              miao:usedMentalIllnessesSchema ?schema .
    ?expert a prov:Person ;
        rdfs:label ?expertLabel .
    ?schema dcterms:title ?schemaTitle .
  }
"""
)

for result in q2:
    print("Expert {} used schema '{}'".format(result[0], result[1]))

Expert Dr. Laura García, PhD used schema 'DSM-5: Diagnostic and Statistical Manual, 5th Edition'


In [5]:
## CQ ##
# What confidence scores do human experts assign to their manual diagnoses?
q3 = ex1_graph.query(
"""
  SELECT DISTINCT ?detectionLabel ?expertLabel  ?confidence
  WHERE {
    ?detection a miao:ManualMentalIllnessesDetection ;
              rdfs:label ?detectionLabel ;
              prov:wasAssociatedWith ?expert ;
              prov:generated ?mentalIllnessesSet .
    ?expert a prov:Person ;
        rdfs:label ?expertLabel .
    ?mentalIllnessesSet a miao:MentalIllnessesSet ;
        miao:hasMentalIllness ?mentalIllness .
    ?mentalIllness a ?mentalIllnessCategory ;
        miao:hasMentalIllnessDetectionConfidence ?confidence .
  }
"""
)

for result in q3:
    print("Detection '{}' from expert '{}' has confidence {}".format(result[0], result[1], result[2]))

Detection 'ManualDetectionGarciaP00010' from expert 'Dr. Laura García, PhD' has confidence 0.82


In [6]:
## CQ ##
#  Which AI models can be used to detect depression?
q4 = ex2_graph.query(
"""
  PREFIX ex2: <https://w3id.org/miao/ex2#>
  SELECT DISTINCT ?modelLabel
  WHERE {
    ?detection a miao:AutomaticMentalIllnessesDetection ;
          mls:executes ?model ;
          prov:generated ?mentalIllnessesSet .
    ?model a mls:Implementation ;
        rdfs:label ?modelLabel .
     ?mentalIllnessesSet a miao:MentalIllnessesSet ;
        miao:hasMentalIllness ?mentalIllness .
    ?mentalIllness a ex2:Depression .
  }
"""
)
for result in q4:
    print("Model {} can be used to detect depression".format(result[0]))

Model PytorchBertModel can be used to detect depression
Model OpenRouterGPT4 can be used to detect depression
Model PytorchMultiModalTransformer can be used to detect depression


In [7]:
## CQ ##
# Which AI models have been applied to detect anxiety from EEG signals?
q5 = ex2_graph.query(
"""
  PREFIX ex2: <https://w3id.org/miao/ex2#>
  SELECT DISTINCT ?modelLabel
  WHERE {
    ?detection a miao:AutomaticMentalIllnessesDetection ;
          mls:executes ?model;
          miao:hasInputData ex2:EEGSignals ;
          prov:generated ?mentalIllnessesSet .
    ?model a mls:Implementation ;
          rdfs:label ?modelLabel .
    ?mentalIllnessesSet a miao:MentalIllnessesSet ;
        miao:hasMentalIllness ?mentalIllness .
    ?mentalIllness a ex2:Anxiety .
  }
"""
)
for result in q5:
    print("Model '{}' has been used to detect anxiety from EEG signals".format(result[0]))


Model 'PytorchLSTMModel' has been used to detect anxiety from EEG signals


In [8]:
## CQ ##
# Which AI models combine multiple data sources?
q6 = ex2_graph.query(
"""
  PREFIX ex2: <https://w3id.org/miao/ex2#>
  SELECT ?modelLabel (COUNT(?data) AS ?numDataSources)
  WHERE {
    ?detection a miao:AutomaticMentalIllnessesDetection ;
          mls:executes ?model;
          miao:hasInputData ?data .
    ?model a mls:Implementation ;
          rdfs:label ?modelLabel .
  }
  GROUP BY ?model ?modelLabel
  HAVING (COUNT(?data) > 1)
"""
)
for result in q6:
    print("Model '{}' combines multiple sources".format(result[0]))



Model 'PytorchMultiModalTransformer' combines multiple sources


In [9]:
## CQ ##
# Which AI models have been applied to detect multiple disorders simultaneously?
q7 = ex2_graph.query(
"""
  SELECT ?modelLabel (COUNT(DISTINCT ?illness) AS ?numDisorders)
  WHERE {
    ?detection a miao:AutomaticMentalIllnessesDetection ;
          mls:executes ?model;
          prov:generated ?mentalIllnessesSet .

    ?mentalIllnessesSet miao:hasMentalIllness ?illness .
    ?model a mls:Implementation ;
          rdfs:label ?modelLabel .
  }
  GROUP BY ?model
  HAVING (COUNT(DISTINCT ?illness) > 1)
"""
)
for result in q7:
    print("Model '{}' have been applied to detect multiple disorder simultaneously".format(result[0]))

Model 'PytorchMultiModalTransformer' have been applied to detect multiple disorder simultaneously


In [10]:
## CQ ##
# Are there AI models that can detect both depression and anxiety from the same data source?
q8 = ex2_graph.query(
"""
  SELECT DISTINCT ?modelLabel
  WHERE {
  ?detection a miao:AutomaticMentalIllnessesDetection ;
         mls:executes ?model;
         miao:hasInputData ?data ;
         prov:generated ?set .
  ?model a mls:Implementation ;
          rdfs:label ?modelLabel .
  ?set miao:hasMentalIllness ?ill1 ;
       miao:hasMentalIllness ?ill2 .
  ?ill1 a ex2:Depression .
  ?ill2 a ex2:Anxiety .
}
"""
)
for result in q8:
    print("Model '{}' can detect depression and anxiety from the same data source".format(result[0]))

Model 'PytorchMultiModalTransformer' can detect depression and anxiety from the same data source


In [11]:
## CQ ##
#  Which model achieved the highest accuracy in predicting depression?
q9 = ex2_graph.query(
"""
  PREFIX ex2: <https://w3id.org/miao/ex2#>
  SELECT ?modelLabel ?score
  WHERE {
    ?detection a miao:AutomaticMentalIllnessesDetection ;
          mls:executes ?model ;
          prov:generated ?mentalIllnessesSet ;
          mls:hasOutput ?evaluation .

    ?model a mls:Implementation ;
          rdfs:label ?modelLabel .

    ?evaluation a mls:ModelEvaluation ;
          mls:specifiedBy ex2:predictiveAccuracy ;
          mls:hasValue ?score .

    ?mentalIllnessesSet miao:hasMentalIllness ?illness .
    ?illness a ex2:Depression .
  }
  ORDER BY DESC(?score)
  LIMIT 1
"""
)
for result in q9:
    print("Model '{}' detected Depression with accuracy of {} ".format(result[0], result[1]))


Model 'OpenRouterGPT4' detected Depression with accuracy of 0.98 
